In [1]:
# Nạp các gói thư viện NumPy và Pandas
import pandas as pd
import numpy as np
import regex
import string

In [2]:
#Tạo đường link
import sys
import os

current = os.path.abspath('')
parent = os.path.dirname(current)
grandparent = os.path.dirname(parent)

In [3]:
#File path
df_file = parent+"/data/processed/input_table.csv"

In [4]:
df = pd.read_csv(df_file)
df = df.dropna().reset_index(drop=True)
df

,new_date,ma_ck,session,return,title_new
0,2023-01-10,BHT,return_in_session,0.000000,tổ_chức đại_hội_đồng cổ_đông thường_niên
1,2023-01-10,HAG,return_before_session,-0.008830,hoàng gia lai hagl lãi trở_lại sau đức hào_phó...
2,2023-01-10,PET,return_in_session,-0.007624,thực_hiện quyền nhận cổ_tức cổ_phiếu
3,2023-01-10,PLX,return_in_session,0.048654,sau khi hạ chỉ tiêu bất_ngờ báo lãi thuế gần
4,2023-01-10,PRT,return_in_session,-0.017544,thông_báo dự_kiến tổ_chức đại_hội_đồng cổ_đông...
...,...,...,...,...,...
538,2023-02-24,SP2,return_in_session,0.000000,bổ_nhiệm bùi quế_lâm giữ chức tgđ
539,2023-02-24,STT,return_in_session,-0.111226,ký_kết hợp_đồng
540,2023-02-24,UDC,return_before_session,0.018562,nghị_quyết hđqt thông_qua phương_án sử_dụng tà...
541,2023-02-24,VOS,return_in_session,-0.004197,tổ_chức đhđcđ thường_niên


In [5]:
# Loai bo nhung dong return la 0
df = df[df['return']!=0]

In [6]:
# Tao bien deep de giu lai nhung dong return thuc su co y nghia
df['deep']=0
df['deep'][df['return']>df['return'].quantile(0.999)]=100
df['deep'][df['return']<df['return'].quantile(0.001)]=100
df = df.reset_index(drop=True)

/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93716/2007535278.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['deep']=0
/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93716/2007535278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['deep'][df['return']>df['return'].quantile(0.999)]=100
/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93716/2007535278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

In [7]:
#Tao bien dummies ma chung khoan
dummies = pd.get_dummies(df['ma_ck'])
dummies

,AGF,AMD,AST,CIG,DHB,DLG,DP2,DXV,HAG,HLG,...,TGG,TNI,TNM,TTF,UDC,VFG,VOS,VSF,VST,VTR
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


# Vectorize

In [18]:
# bag of words
from sklearn.feature_extraction.text import CountVectorizer

X = df['title_new']
y = df['return']

In [19]:
vectorizer = CountVectorizer()
vectorizer.fit(X)
X_vec = vectorizer.transform(X)
X_vec

<271x622 sparse matrix of type '<class 'numpy.int64'>'
	with 2175 stored elements in Compressed Sparse Row format>

In [20]:
tfidf_feature_names = vectorizer.get_feature_names_out()
tfidf_feature_names

array(['agrico', 'airlines', 'an', 'an_khang', 'ava', 'avasprort', 'ban',
       'ban_hành', 'bapi', 'bctc', 'biên', 'biến_động', 'biểu_quyết',
       'biện_pháp', 'bks', 'bluetronics', 'buộc', 'bàn', 'bách_hoá',
       'bán', 'bán_lẻ', 'bánh_chưng', 'báo', 'báo_chí', 'báo_cáo', 'bên',
       'bí_mật', 'bđs', 'bĩ_cực', 'bước', 'bản_án', 'bảng', 'bảo',
       'bảo_lãnh', 'bất_ngờ', 'bất_thường', 'bất_động_sản', 'bầu', 'bắc',
       'bắt', 'bị', 'bố_trí', 'bố_vợ', 'bốc', 'bổ_nhiệm', 'bổ_sung', 'ca',
       'cam_kết', 'cambodia', 'cao', 'cao_cấp', 'cao_ngất_ngưởng',
       'cao_tốc', 'capital', 'cbtt', 'chi_nhánh', 'chi_trả', 'chia',
       'chuối', 'chuỗi', 'chào', 'chênh_lệch', 'chính_sách', 'chính_thức',
       'chưa', 'chấm_dứt', 'chấp_thuận', 'chậm', 'chỉ', 'chỉ_tiêu',
       'chốt', 'chợ', 'chủ', 'chủ_trương', 'chủ_tịch', 'chức',
       'chức_danh', 'chức_vụ', 'chứng_khoán', 'chứng_nhận', 'ck', 'clb',
       'con', 'cp', 'ct', 'ctcp', 'cty', 'cung_cấp', 'cuối', 'cuối_cùng',
       '

In [21]:
# Tao bang sentiment gom nhung tu da duoc vectorize
sentiments = pd.DataFrame(data=X_vec.toarray(), columns = tfidf_feature_names)
sentiments[dummies.columns] = dummies

In [22]:
sentiments['deep'] = df['deep']

In [23]:
#Nhung tu ngu co anh huong lon
deep = pd.DataFrame(data = sentiments[sentiments['deep']==100].sum(), columns=['deep'])
deep = deep[deep['deep']>0]
deep_lst = deep.index.tolist()
deep_lst

['bán',
 'báo_cáo',
 'chủ_tịch',
 'công_ty',
 'hđqt',
 'nguyễn_văn_cưng',
 'quản_trị',
 'đã',
 'HLY',
 'TNM',
 'deep']

In [24]:
# Top những từ xuất hiện nhiều hơn 1000 lần
big = pd.DataFrame(data=np.count_nonzero(sentiments, axis=0), index=sentiments.columns, columns=['counts'])
big_lst = big[big['counts']>0.01*len(sentiments)].index.tolist()
big_lst

['bctc',
 'biến_động',
 'biện_pháp',
 'bán',
 'báo',
 'báo_chí',
 'báo_cáo',
 'bên',
 'bước',
 'bất_thường',
 'bị',
 'bổ_nhiệm',
 'cao',
 'cao_ngất_ngưởng',
 'cbtt',
 'chuối',
 'chuỗi',
 'chào',
 'chênh_lệch',
 'chưa',
 'chậm',
 'chốt',
 'chủ_trương',
 'chủ_tịch',
 'chức',
 'chứng_khoán',
 'chứng_nhận',
 'cp',
 'ctcp',
 'cty',
 'cuối_cùng',
 'có',
 'công_bố',
 'công_ty',
 'cùng',
 'cảnh_báo',
 'cổ_phiếu',
 'cổ_phần',
 'cổ_tức',
 'cổ_đông',
 'danh_sách',
 'di_động',
 'diện',
 'dn',
 'doanh_nghiệp',
 'doanh_thu',
 'dự_án',
 'gia',
 'giao_dịch',
 'giá',
 'giải_trình',
 'giảm',
 'giấy',
 'giờ',
 'giữ',
 'gây',
 'gấp',
 'gần',
 'gốc',
 'hagl',
 'hn',
 'hoàng',
 'hoạt_động',
 'huỷ',
 'hàng',
 'hđqt',
 'hơn',
 'họp',
 'hợp_nhất',
 'hợp_đồng',
 'hủy',
 'khi',
 'khí',
 'khả_năng',
 'khắc_phục',
 'kinh_doanh',
 'kiểm_soát',
 'kqkd',
 'ký_kết',
 'kế',
 'kế_hoạch',
 'kết_quả',
 'kỳ',
 'kỷ_lục',
 'lai',
 'liên_quan',
 'lnst',
 'long',
 'là',
 'lãi',
 'lãi_ròng',
 'lô',
 'lưu_ý',
 'lại',
 'lỗ',
 'lộ

In [25]:
for i in deep_lst:
  if i not in big_lst:
    big_lst.append(i)

In [26]:
big_lst

['bctc',
 'biến_động',
 'biện_pháp',
 'bán',
 'báo',
 'báo_chí',
 'báo_cáo',
 'bên',
 'bước',
 'bất_thường',
 'bị',
 'bổ_nhiệm',
 'cao',
 'cao_ngất_ngưởng',
 'cbtt',
 'chuối',
 'chuỗi',
 'chào',
 'chênh_lệch',
 'chưa',
 'chậm',
 'chốt',
 'chủ_trương',
 'chủ_tịch',
 'chức',
 'chứng_khoán',
 'chứng_nhận',
 'cp',
 'ctcp',
 'cty',
 'cuối_cùng',
 'có',
 'công_bố',
 'công_ty',
 'cùng',
 'cảnh_báo',
 'cổ_phiếu',
 'cổ_phần',
 'cổ_tức',
 'cổ_đông',
 'danh_sách',
 'di_động',
 'diện',
 'dn',
 'doanh_nghiệp',
 'doanh_thu',
 'dự_án',
 'gia',
 'giao_dịch',
 'giá',
 'giải_trình',
 'giảm',
 'giấy',
 'giờ',
 'giữ',
 'gây',
 'gấp',
 'gần',
 'gốc',
 'hagl',
 'hn',
 'hoàng',
 'hoạt_động',
 'huỷ',
 'hàng',
 'hđqt',
 'hơn',
 'họp',
 'hợp_nhất',
 'hợp_đồng',
 'hủy',
 'khi',
 'khí',
 'khả_năng',
 'khắc_phục',
 'kinh_doanh',
 'kiểm_soát',
 'kqkd',
 'ký_kết',
 'kế',
 'kế_hoạch',
 'kết_quả',
 'kỳ',
 'kỷ_lục',
 'lai',
 'liên_quan',
 'lnst',
 'long',
 'là',
 'lãi',
 'lãi_ròng',
 'lô',
 'lưu_ý',
 'lại',
 'lỗ',
 'lộ

In [27]:
sentiments_new = sentiments[big_lst]
sentiments_new['title'] = df['title_new']
sentiments_new['return'] = df['return']
sentiments_new['ma_ck'] = df['ma_ck']
sentiments_new['Date'] = df['new_date']
sentiments_new['session'] = df['session']
sentiments_new['sum'] = sentiments_new[big_lst].sum(axis=1)
sentiments_new = sentiments_new[sentiments_new['sum']>sentiments_new['sum'].quantile(0.2)]
sentiments_new

/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93716/990197602.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiments_new['title'] = df['title_new']
/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93716/990197602.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiments_new['return'] = df['return']
/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93716/990197602.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

,bctc,biến_động,biện_pháp,bán,báo,báo_chí,báo_cáo,bên,bước,bất_thường,...,nguyễn_văn_cưng,HLY,TNM,deep,title,return,ma_ck,Date,session,sum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,hoàng gia lai hagl lãi trở_lại sau đức hào_phó...,-0.008830,HAG,2023-01-10,return_before_session,12
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,thực_hiện quyền nhận cổ_tức cổ_phiếu,-0.007624,PET,2023-01-10,return_in_session,6
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,sau khi hạ chỉ tiêu bất_ngờ báo lãi thuế gần,0.048654,PLX,2023-01-10,return_in_session,7
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,thông_báo dự_kiến tổ_chức đại_hội_đồng cổ_đông...,-0.017544,PRT,2023-01-10,return_in_session,6
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,thông_qua chủ_trương giao_dịch bên có liên_quan,-0.065032,TDH,2023-01-10,return_before_session,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,chuỗi dự_án khí điện lô b tiến thêm bước quan_...,-0.002611,PLX,2023-02-24,return_before_session,12
265,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,chuỗi dự_án khí điện lô b tiến thêm bước quan_...,0.002291,PVD,2023-02-24,return_before_session,12
266,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,chuỗi dự_án khí điện lô b tiến thêm bước quan_...,0.003766,PVS,2023-02-24,return_before_session,12
267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,thông_báo dự_kiến tổ_chức đại_hội_đồng cổ_đông...,0.022990,PVV,2023-02-24,return_in_session,10


In [28]:
for i in sentiments_new.columns:
  if '.1' in i:
    print(i)

In [29]:
# Save file
sentiments_new.to_csv(parent+'/data/processed/sentiments_vector.csv', index=False)